# Word2Vec 词向量
将字词从文字空间映射到向量空间。  
Map words from text space to vector space.

每个字词都会有一个对应的代表其__语义__的向量。  
Each word has a corresponding vector which represents the __semantics__ of a word.

有多重方法做到这一点。  
Many methods can achieve such a goal.

有基于次数的方法。  
There are count based methods.

本教程介绍的是论文《[Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)》中所使用的方法。这可以被视作为一种概率式方法。  
This tutorial explains the method used in [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf). This can be seen as a probabilistic method.

__为了最大地照顾到各种同学(不会中文的同学)，我主要用英文。关键句段会有中文。__  
__For maximum fellow learners reach（non-Chinese learners）, I will mostly use English. I will provide Chinese for some key phrases.__

__英文是在学界与开发者社区是一门重要的语言。中国的同学们多看英文资料也是有好处的。__  
__English is an important languge in academia and developer communities. Fellow learners in China can benefit a lot by reading English materials.__

### 预备知识 Pre-request:
本教程是编程向的。2分理论，8分编程。  
This tutorial is programming oriented. I will descripe it as 20% theory and 80% programming.

所以，我假定你懂得基本的深度学习。  
Therefore, I assume you already have understand the basics of __Deep Learning__.

我也假定你有一定的TensorFlow基础。  
I also assume that you have some experience with [__TensorFlow__](https://www.tensorflow.org/).

我强烈建议你在继续本教程之前，阅读文末提供的链接。  
I strongly encourage you to read all materials linked at the end of this notebook.

#### To be specific, you ought to know:
__1. One-hot encoding__  
__2. Cross entropy loss__  
__3. Maximum likelihood estimation__  
__4. Softmax classifier__  
__5. Fully connected neural network__  

## A TensorFlow Example
This is based on [TensorFlow website tutorial](https://www.tensorflow.org/tutorials/word2vec)

In [1]:
### Python3

# First let's import all the necessary dependencies
import collections
import math
import os
import random
import zipfile

import numpy as np
import urllib
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

from pandas import DataFrame

# Check TensorFlow version. I encourage you to use 1.X. I use 1.0
print(tf.__version__)

1.0.0


In [6]:
# Step 1: Download the data.
url = 'http://mattmahoney.net/dc/'

# We want to download data from this url, if it's the first time
def maybe_download(filename, expected_bytes):
  """
  Download a file if not present, and make sure it's the right size.
  """
  if not os.path.exists(filename):
    filename, _ = urllib.request.urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', filename)
  else:
    print(statinfo.st_size)
    raise Exception('Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)  # do not change this line


# Read the data into a list of strings.
def read_data(filename):
  """Extract the first file enclosed in a zip file as a list of words"""
  with zipfile.ZipFile(filename) as f:
    data = tf.compat.as_str(f.read(f.namelist()[0])).split()
  return data

# Now we have the data
words = read_data(filename)
print('Data size', len(words))  # The length of "words", not bytes

Found and verified text8.zip
Data size 17005207


In [7]:
# Step 2: Build the dictionary and replace rare words with UNK token.

# We only want 50000 words in the vocabulary just for the sake of demonstration
vocabulary_size = 50000


def build_dataset(words, vocabulary_size):
  # UNK is a special token we use to represent "empty word", or thrown away words
  # Since we want the most common 49999 words in the data
  # we will throw away some less commonly used words
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count += 1
    data.append(index)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words, vocabulary_size)

# Python uses reference counting as its memory management method.
# delete a name from a namespace will descrease the count of references to an object by one
# since we only have 1 reference to the underlaying "word" object
# deleting the name "word" will get rid of all references to that object
# then this object will be freed from memory
del words  # Hint to reduce memory.

# Reference counting is different from gabage collection, which is used by JVM for example.

# Gabage collection happens when the runtime schedules to do so.
# When gabage collection happens is beyond the control of programmers.
# But, in general, gabage collection is faster than reference counting.

# Reference counting is slower because it happens every time when a refernece is created or deleted
# But it offers programmers more control.

print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

data_index = 0

Most common words (+UNK) [['UNK', 418391], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764)]
Sample data [5241, 3081, 12, 6, 195, 2, 3134, 46, 59, 156] ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against']


In [9]:
# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1  # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  for i in range(batch_size // num_skips):
    target = skip_window  # target label at the center of the buffer
    targets_to_avoid = [skip_window]
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[target]
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  # Backtrack a little bit to avoid skipping words in the end of a batch
  data_index = (data_index + len(data) - span) % len(data)
  return batch, labels

batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
  print(batch[i], reverse_dictionary[batch[i]], '->', labels[i, 0], reverse_dictionary[labels[i, 0]])

3081 originated -> 5241 anarchism
3081 originated -> 12 as
12 as -> 6 a
12 as -> 3081 originated
6 a -> 12 as
6 a -> 195 term
195 term -> 2 of
195 term -> 6 a


In [11]:
# Step 4: Build and train a skip-gram model.
batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.


graph = tf.Graph()
with graph.as_default():

  # Input data.
  train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
    embeddings = tf.Variable(
      tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
      tf.truncated_normal([vocabulary_size, embedding_size],
                          stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
  loss = tf.reduce_mean(
    tf.nn.nce_loss(weights=nce_weights,
                   biases=nce_biases,
                   labels=train_labels,
                   inputs=embed,
                   num_sampled=num_sampled,
                   num_classes=vocabulary_size))

  # Construct the SGD optimizer using a learning rate of 1.0.
  optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

  # Compute the cosine similarity between minibatch examples and all embeddings.
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
  similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)

  # Add variable initializer.
  init = tf.global_variables_initializer()

In [ ]:
# Step 5: Begin training.
num_steps = 100001
LOG_DIR = './log/'

with tf.Session(graph=graph) as session:
  # We must initialize all variables before we use them.
  init.run()
  print("Initialized")

  average_loss = 0
  for step in xrange(num_steps):
    batch_inputs, batch_labels = generate_batch(
      batch_size, num_skips, skip_window)
    feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
    _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += loss_val

    if step % 2000 == 0:
      if step > 0:
        average_loss /= 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print("Average loss at step ", step, ": ", average_loss)
      average_loss = 0


  """
  Use TensorBoard to visualize our model. 
  This is not included in the TensorFlow website tutorial.
  """
  words_to_visualize = 3000
  final_embeddings = normalized_embeddings.eval()[:words_to_visualize]
  embedding_var = tf.Variable(final_embeddings)
  session.run(embedding_var.initializer)
  saver = tf.train.Saver([embedding_var])
  saver.save(session, os.path.join(LOG_DIR, "model.ckpt"), 0)

  # Format: tensorflow/contrib/tensorboard/plugins/projector/projector_config.proto
  config = projector.ProjectorConfig()

  # You can add multiple embeddings. Here we add only one.
  embedding = config.embeddings.add()
  embedding.tensor_name = embedding_var.name
  # Link this tensor to its metadata file (e.g. labels).
  embedding.metadata_path = os.path.join(LOG_DIR, 'metadata.tsv')

  # Use the same LOG_DIR where you stored your checkpoint.
  summary_writer = tf.summary.FileWriter(LOG_DIR)
  summary_writer.add_graph(graph)

  # The next line writes a projector_config.pbtxt in the LOG_DIR. TensorBoard will
  # read this file during startup.
  projector.visualize_embeddings(summary_writer, config)

  # Write the metadata file to disk so that TensorBoard can find it later
  labels = [(reverse_dictionary[i], i) for i in range(words_to_visualize)]
  DataFrame(labels, columns=['word', 'freq_rank']).to_csv('log/metadata.tsv', index=False, sep='\t')


## A Gensim Example
Besides the [TensorFlow example](word2vec.py), I show you a Gensim example here.

[Gensim](https://radimrehurek.com/gensim/) is a topic modelling toolkit. It has word2vec implemented.

In [5]:
### First, get the data
### We use the complete works of Shakespear



## References
### Paper
[Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)  
[Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/pdf/1310.4546.pdf)  


### Other Tutorials
[A post by Chris McCormick](http://mccormickml.com/2016/04/27/word2vec-resources/)  
[Second post by Chris McCormick](http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/)  

### TensorFlow
[TensorFlow Tutorial](https://www.tensorflow.org/tutorials/word2vec)  
[TensorFlow Example Code](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/word2vec/word2vec_basic.py)  